# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [6]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [7]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [8]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [9]:
data_by_CustomerID_Product=data.groupby(by=['CustomerID','ProductName'])['Quantity'].sum()
print(data_by_CustomerID_Product)

CustomerID  ProductName                  
33          Apricots - Dried                  1
            Assorted Desserts                 1
            Bandage - Flexible Neon           1
            Bar Mix - Pina Colada, 355 Ml     1
            Beans - Kidney, Canned            1
                                             ..
98200       Vol Au Vents                     50
            Wasabi Powder                    25
            Wine - Fume Blanc Fetzer         25
            Wine - Hardys Bankside Shiraz    25
            Yogurt - French Vanilla          25
Name: Quantity, Length: 63628, dtype: int64


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [10]:
Pivot_Table=pd.pivot_table(data,values='Quantity',index='ProductName',columns='CustomerID',aggfunc='sum').fillna(0)
Pivot_Table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
distances = pd.DataFrame(1/(1 + squareform(pdist(Pivot_Table.T, 'euclidean'))),index=Pivot_Table.columns, columns=Pivot_Table.columns)

print(distances)

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

CustomerID     477       639       64

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [12]:
similarities = distances[639].sort_values(ascending=False)[1:]
similarities.head()

CustomerID
3317    0.088530
3535    0.087779
3074    0.087047
3305    0.086333
649     0.085983
Name: 639, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
Clientes = data_by_CustomerID_Product.loc[list(similarities.index)]
Clientes



CustomerID  ProductName                  
33          Apricots - Dried                  1
            Assorted Desserts                 1
            Bandage - Flexible Neon           1
            Bar Mix - Pina Colada, 355 Ml     1
            Beans - Kidney, Canned            1
                                             ..
98200       Vol Au Vents                     50
            Wasabi Powder                    25
            Wine - Fume Blanc Fetzer         25
            Wine - Hardys Bankside Shiraz    25
            Yogurt - French Vanilla          25
Name: Quantity, Length: 63570, dtype: int64

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [14]:
Order_sim_quantity=Clientes.groupby(by=['ProductName']).sum().sort_values(ascending=False)
Order_sim_quantity

ProductName
Cookies - Assorted             2403
Flavouring - Orange            2396
Fenngreek Seed                 2390
Wine - White Cab Sauv.on       2387
Bandage - Flexible Neon        2347
                               ... 
Scallops 60/80 Iqf             1546
Rice - Long Grain              1531
Kellogs Special K Cereal       1496
Rabbit - Whole                 1345
Water - Green Tea Refresher    1322
Name: Quantity, Length: 452, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [15]:
merging=Pivot_Table.merge(Order_sim_quantity,on='ProductName')
recomended_products = merging[merging[639]==0].nlargest(5,'Quantity')['Quantity']
recomended_products

ProductName
Cookies - Assorted                  2403
Wine - White Cab Sauv.on            2387
Bandage - Flexible Neon             2347
Beef - Montreal Smoked Brisket      2344
Oil - Shortening - All - Purpose    2344
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [17]:
recomendations = {}
distances = pd.DataFrame(1/(1 + squareform(pdist(Pivot_Table.T, 'euclidean'))),index=Pivot_Table.columns, columns=Pivot_Table.columns)

    


In [42]:
for elem in Pivot_Table.columns:
    
    similars = distances[elem][1:].nlargest(5)
    
    clients = data_by_CustomerID_Product.loc[list(similars.index)]   
    
    products = (clients.groupby('ProductName').sum()).sort_values(ascending=False)
    
    merged = Pivot_Table.merge(products, on = 'ProductName')
    
    recomendations[elem] = list((merged[merged[elem]==0].nlargest(5,'Quantity')['Quantity']).index)
recomendations

{33: ['Butter - Unsalted',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein',
  'Wine - Ej Gallo Sierra Valley',
  'Bacardi Breezer - Tropical'],
 200: ['Muffin - Carrot Individual Wrap',
  'Pork - Kidney',
  'Soup - Campbells Bean Medley',
  'Beef Wellington',
  'Cheese - Boursin, Garlic / Herbs'],
 264: ['Fish - Scallops, Cold Smoked',
  'Bar - Granola Trail Mix Fruit Nut',
  'Beef - Inside Round',
  'Beef - Striploin Aa',
  'Beef - Top Sirloin'],
 356: ['Veal - Inside, Choice',
  'Butter - Unsalted',
  'Beans - Wax',
  'Beef - Inside Round',
  'Beef - Rib Eye Aaa'],
 412: ['Pepper - Black, Whole',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein',
  'Beef Wellington',
  'Butter - Unsalted'],
 464: ['Butter - Unsalted',
  'Bar - Granola Trail Mix Fruit Nut',
  'Pepper - Black, Whole',
  'Sausage - Breakfast',
  'Yoghurt Tubes'],
 477: ['Pepper - Black, Whole',
  'Wine - Blue Nun Qualitatswein',
  'Butter - Unsalted',
  'Cheese - Bocconcini',
  'Che

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [44]:
recommendation_data = pd.DataFrame.from_dict(recomendations, orient = 'columns')
recommendation_data

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Muffin - Carrot Individual Wrap,"Fish - Scallops, Cold Smoked","Veal - Inside, Choice","Pepper - Black, Whole",Butter - Unsalted,"Pepper - Black, Whole",Cod - Black Whole Fillet,Appetizer - Mushroom Tart,"Pepper - Black, Whole",...,Meldea Green Tea Liquor,Papayas,"Pork - Loin, Bone - In",Ice Cream Bar - Oreo Cone,Rum - Mount Gay Eclipes,"Soup - Campbells, Lentil",Skirt - 29 Foot,Ice Cream Bar - Oreo Cone,Crackers - Trio,Sherry - Dry
1,Soup - Campbells Bean Medley,Pork - Kidney,Bar - Granola Trail Mix Fruit Nut,Butter - Unsalted,Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,Wine - Blue Nun Qualitatswein,Wine - Ej Gallo Sierra Valley,Bar - Granola Trail Mix Fruit Nut,"Veal - Inside, Choice",...,Muffin - Carrot Individual Wrap,Muffin Batt - Choc Chk,Cake - Box Window 10x10x2.5,Coffee - Dark Roast,"Placemat - Scallop, White","Cheese - Brie,danish",Milk - 1%,Halibut - Steaks,Pernod,Beans - Wax
2,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,Beef - Inside Round,Beans - Wax,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Butter - Unsalted,"Pepper - Black, Whole",Anchovy Paste - 56 G Tube,Anchovy Paste - 56 G Tube,...,Coffee - Dark Roast,Mussels - Frozen,Coffee Decaf Colombian,Dc Hikiage Hira Huba,Tofu - Firm,Pants Custom Dry Clean,Yoghurt Tubes,"Lamb - Whole, Fresh",Cream Of Tartar,Water - Spring Water 500ml
3,Wine - Ej Gallo Sierra Valley,Beef Wellington,Beef - Striploin Aa,Beef - Inside Round,Beef Wellington,Sausage - Breakfast,Cheese - Bocconcini,"Wine - Magnotta, Merlot Sr Vqa",Bandage - Fexible 1x3,Bread - Italian Roll With Herbs,...,Cheese - Wine,Coffee - Dark Roast,Curry Paste - Madras,Garlic - Peeled,"Cheese - Boursin, Garlic / Herbs",Wiberg Super Cure,Soupfoamcont12oz 112con,Lettuce - Spring Mix,Lettuce - Spring Mix,Beef - Prime Rib Aaa
4,Bacardi Breezer - Tropical,"Cheese - Boursin, Garlic / Herbs",Beef - Top Sirloin,Beef - Rib Eye Aaa,Butter - Unsalted,Yoghurt Tubes,Cheese - Cambozola,Bandage - Fexible 1x3,Beef - Top Sirloin - Aaa,Butter - Unsalted,...,Rice - Long Grain,Longos - Chicken Wings,Fenngreek Seed,Sun - Dried Tomatoes,Sobe - Tropical Energy,Shrimp - 31/40,Onions - Vidalia,Chicken - Wieners,Pastry - Choclate Baked,Cod - Black Whole Fillet


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [45]:
recomendations = {}
distances = pd.DataFrame(1/(1 + squareform(pdist(Pivot_Table.T, 'jaccard'))),index=Pivot_Table.columns, columns=Pivot_Table.columns)
for elem in Pivot_Table.columns:
    
    similars = distances[elem][1:].nlargest(5)
    
    clients = data_by_CustomerID_Product.loc[list(similars.index)]   
    
    products = (clients.groupby('ProductName').sum()).sort_values(ascending=False)
    
    merged = Pivot_Table.merge(products, on = 'ProductName')
    
    recomendations[elem] = list((merged[merged[elem]==0].nlargest(5,'Quantity')['Quantity']).index)
recomendations
recommendation_data = pd.DataFrame.from_dict(recomendations, orient = 'columns')

In [46]:
recommendation_data

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Loquat,Muffin - Carrot Individual Wrap,Flavouring - Orange,Beef - Inside Round,Chocolate - Dark,Cheese - Mozzarella,Ice Cream Bar - Hageen Daz To,Bacardi Breezer - Tropical,"Chestnuts - Whole,canned",Cod - Black Whole Fillet,...,"Beef - Ground, Extra Lean, Fresh",Spice - Peppercorn Melange,Coffee - Irish Cream,Bacardi Breezer - Tropical,Fenngreek Seed,Banana Turning,Cheese - Parmesan Cubes,Chocolate - Compound Coating,Coffee - Irish Cream,Beef - Top Sirloin
1,Bananas,"Beans - Kidney, Canned",Sauce - Demi Glace,Sardines,Guinea Fowl,Sausage - Breakfast,Beer - Blue,Soup Knorr Chili With Beans,Tea - Jasmin Green,Beans - Wax,...,Flour - Whole Wheat,Apricots - Halves,Fenngreek Seed,"Beer - Alexander Kieths, Pale Ale",Bread - Italian Corn Meal Poly,"Beans - Kidney, Canned",Bread - French Baquette,Mustard Prepared,Soup V8 Roasted Red Pepper,Browning Caramel Glace
2,Blueberries,Chips Potato Salt Vinegar 43g,Beer - Blue,"Tart Shells - Sweet, 4",Lettuce - Spring Mix,"Chestnuts - Whole,canned",General Purpose Trigger,"Beef - Ground, Extra Lean, Fresh",Appetizer - Sausage Rolls,Appetizer - Sausage Rolls,...,Langers - Ruby Red Grapfruit,Bread - French Baquette,Garlic,"Cheese - Brie,danish","Chocolate - Semi Sweet, Calets",Cocoa Butter,Flour - Teff,Assorted Desserts,Blackberries,Muffin - Carrot Individual Wrap
3,"Cheese - Brie, Triple Creme",Pork - Kidney,Bread - Italian Corn Meal Poly,"Veal - Inside, Choice","Pepper - Black, Whole",Juice - V8 Splash,Juice - Apple Cider,Phyllo Dough,Bouq All Italian - Primerba,Bacardi Breezer - Tropical,...,Sobe - Tropical Energy,Bread - Multigrain,Muffin - Carrot Individual Wrap,"Garlic - Primerba, Paste",Crackers Cheez It,Ecolab - Lime - A - Way 4/4 L,Rice - Jasmine Sented,Banana - Leaves,Crab - Imitation Flakes,Mussels - Cultivated
4,Cheese Cloth No 100,Rice - Jasmine Sented,Chocolate - Compound Coating,Bay Leaf,"Tart Shells - Sweet, 4",Pomello,"Lamb - Pieces, Diced",Sauerkraut,Bread - Hot Dog Buns,Bandage - Flexible Neon,...,Veal - Eye Of Round,Bread Crumbs - Panko,"Rum - Coconut, Malibu",Quiche Assorted,Pastry - Choclate Baked,Fenngreek Seed,Scampi Tail,Sprouts - Baby Pea Tendrils,Muffin - Carrot Individual Wrap,Turkey - Oven Roast Breast
